In [1]:
#Purpose of this code is to do k Nearest neighbors on data set of songs 

#IMPORTANT: input personal client id and client secret for code to work
#IMPORTANT: may have to change dataset path to be accurate

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import spotipy

from spotipy.oauth2 import SpotifyOAuth
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='client_id',
                                               client_secret='client_secret',
                                               redirect_uri='http://localhost:8080/callback',
                                               scope='playlist-read-private'))

#Load 1000 playlist dataset
dataset = json.load(open('playlist_data/sampledata/mpd.slice.0-999.json'))
df = pd.DataFrame(dataset["playlists"])


In [2]:
#make list of all songs in all playlists, give each song the corresponding playlist id so training can work on all playlists at once
all_songs = []
for i in df.index:
    for j in range(df.num_tracks[i]):
        all_songs.append(
            df.tracks[i][j]
        )
        df.tracks[i][j]['pid'] = df['pid'][i]
all_songs_df = pd.DataFrame(all_songs)

In [ ]:
#create audio features dataframe that has the corresponding playlist id with we will use for classifying
aud_feat = pd.DataFrame(sp.audio_features(all_songs_df.head(100)['track_uri']))
aud_feat['pid']=all_songs_df['pid']

In [ ]:
#train_test_split
X = aud_feat[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']]
y = aud_feat.pid
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1,stratify=y)

In [6]:
#implement kNN, choosing k is important
knn = KNeighborsClassifier(4)

knn.fit(X_train,y_train)

#predict on training set
knn.predict(X_test)

#check accuracy of model on test data
knn.score(X_test,y_test)

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
      dtype=int64)